In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import import_ipynb
import sys
import importlib
import csv

from datetime import datetime, timedelta

from Data_Utilities import load_csv, dataFrameSave
from Args import statisticsExtractionsArguments


def addLabels(x,y,percentage=False):
     for i in range(len(x)):
        if percentage:
            plt.text(i, y[i]+1, str(round(y[i],2))+"%", ha = 'center')
        else:
            plt.text(i, y[i]+1, round(y[i],2), ha = 'center')

def getPercentageValues(totals):
    
    total = sum(totals)
    result = []
    for t in totals:
        result.append((t/total)*100)
    
    return result
            

In [ ]:
def averageRpEmotions(path, RP, Location="?"):
    csv = load_csv(path)
    headers = csv[0]
    data = csv[1:]
    
    df = pd.DataFrame(data, columns = headers)
    
    if RP == "Start":
        rpRows = df.loc[(df['RP'] == "RP1") | (df['RP'] == "RP2")]
    elif RP == "Mountain":
        rpRows = df.loc[(df['RP'] == "RP3") | (df['RP'] == "RP4") | (df['RP'] == "RP5") | (df['RP'] == "RP6")]
    elif RP == "Rock":
        rpRows = df.loc[(df['RP'] == "RP7") | (df['RP'] == "RP8")]
    elif RP == "End":
        rpRows = df.loc[(df['RP'] == "RP9") | (df['RP'] == "RP11") | (df['RP'] == "RP13")]
    else:
        rpRows = df.loc[df['RP'] == RP]
    
    cummulative = [0,0,0,0,0,0,0,0]
    for index, row in rpRows.iterrows():
        cummulative[0] = cummulative[0]+1
        cummulative[1] = cummulative[1]+float(row["Anger"])
        cummulative[2] = cummulative[2]+float(row["Disgust"])
        cummulative[3] = cummulative[3]+float(row["Fear"])
        cummulative[4] = cummulative[4]+float(row["Happiness"])
        cummulative[5] = cummulative[5]+float(row["Sadness"])
        cummulative[6] = cummulative[6]+float(row["Surprise"])
        cummulative[7] = cummulative[7]+float(row["Neutral"])
        
    result = [(cummulative[1]/cummulative[0])*100,
              (cummulative[2]/cummulative[0])*100,
              (cummulative[3]/cummulative[0])*100,
              (cummulative[4]/cummulative[0])*100,
              (cummulative[5]/cummulative[0])*100,
              (cummulative[6]/cummulative[0])*100,
              (cummulative[7]/cummulative[0])*100]
    
    print("Sample total =",cummulative[0])
    plt.bar(rpRows.columns[2:],result)
    addLabels(result, result,True)
    plt.xlabel("Emotions")
    plt.ylabel("Percentage")
    plt.yticks(np.arange(0, 110, 10))
    title = "Average "+RP+ " "+Location
    plt.title(title)
    plt.savefig("A"+RP+".png")
    plt.show()
    
    return result
    
def averageEmotions(path, RPNumberLocations):
    
    answer = input("Do you want ro group RPS?(y)")
    
    if answer != "y":
        rpAverages = []
        for key in RPNumberLocations:
            rpAverages.append(averageRpEmotions(path, "RP"+str(key), RPNumberLocations[key]))
        showAverageEvolution(rpAverages)
    else:
        groupAverages = []
        groupNames = ["Start","Mountain","Rock","End"]
        for key in groupNames:
            groupAverages.append(averageRpEmotions(path, key, ""))
        showAverageEvolution(groupAverages, rpNames=groupNames)
        
def showAverageEvolution(averages, emotionsNames = ["anger","disgust","fear","happiness","sadness","surprise","neutral"],
                        rpNames = ("RP1","RP2","RP3","RP4","RP5","RP6","RP7","RP8","RP9","RP11","RP13")):
    
    emotionsAverages =[]
    for a in range(len(emotionsNames)):
        current = [i[a] for i in averages]
        emotionsAverages.append(current)
        plt.title(emotionsNames[a]+" Evolution")
        plt.plot(rpNames,current)
        addLabels(current, current,True)
        plt.yticks(np.arange(0, 110, 10))
        plt.savefig(emotionsNames[a]+" Evolution"+".png")
        plt.show()
        
    plt.title("Emotions evolution")
    for ea in range(len(emotionsNames)):
        plt.plot(rpNames,emotionsAverages[ea], label = emotionsNames[ea])
    
    plt.yticks(np.arange(0, 110, 10))
    plt.legend(loc=0,fontsize='small')
    
    plt.savefig("Emotions evolution.png")
    plt.show()
    


In [ ]:
def maxEmotions(path, RPNumberLocations):
    
    answer = input("Do you want ro group RPS?(y)")
    
    if answer != "y":
        for key in RPNumberLocations:
            maxRPEmotions(path, "RP"+str(key), RPNumberLocations[key])
    else:
        groupNames = ["Start","Mountain","Rock","End"]
        for key in groupNames:
            maxRPEmotions(path, key, "")
        
def maxRPEmotions(path, RP, Location="?"):
    csv = load_csv(path)
    headers = csv[0]
    data = csv[1:]
    
    df = pd.DataFrame(data, columns = headers) 
    if RP == "Start":
        rpRows = df.loc[(df['RP'] == "RP1") | (df['RP'] == "RP2")]
    elif RP == "Mountain":
        rpRows = df.loc[(df['RP'] == "RP3") | (df['RP'] == "RP4") | (df['RP'] == "RP5") | (df['RP'] == "RP6")]
    elif RP == "Rock":
        rpRows = df.loc[(df['RP'] == "RP7") | (df['RP'] == "RP8")]
    elif RP == "End":
        rpRows = df.loc[(df['RP'] == "RP9") | (df['RP'] == "RP11") | (df['RP'] == "RP13")]
    else:
        rpRows = df.loc[df['RP'] == RP]
    
    maxTotals = [0,0,0,0,0,0,0]
    currentRow = [0,0,0,0,0,0,0]
    for index, row in rpRows.iterrows():
        currentRow[0] = float(row["Anger"])
        currentRow[1] = float(row["Disgust"])
        currentRow[2] = float(row["Fear"])
        currentRow[3] = float(row["Happiness"])
        currentRow[4] = float(row["Sadness"])
        currentRow[5] = float(row["Surprise"])
        currentRow[6] = float(row["Neutral"])
        maxTotals[currentRow.index(max(currentRow))] = maxTotals[currentRow.index(max(currentRow))]+max(currentRow)
    
    percentageTotal = getPercentageValues(maxTotals)
    
    plt.bar(rpRows.columns[2:],percentageTotal)
    addLabels(percentageTotal, percentageTotal,True)
    plt.yticks(np.arange(0, 110, 10))
    plt.xlabel("Emotions")
    plt.ylabel("Total")
    title = "Max "+RP+" "+Location+" "+str(len(rpRows)) + " samples"
    plt.title(title)
    plt.savefig("M"+RP+".png")
    plt.show()

In [ ]:
def totalEmotions(path, RPNumberLocations):
    answer = input("Do you want ro group RPS?(y)")
    
    if answer != "y":
        for key in RPNumberLocations:
            totalRPEmotions(path, "RP"+str(key), RPNumberLocations[key])

    else:
        groupNames = ["Start","Mountain","Rock","End"]
        for key in groupNames:
            totalRPEmotions(path, key, "")
        
def totalRPEmotions(path, RP, Location="?"):
    csv = load_csv(path)
    headers = csv[0]
    data = csv[1:]
    
    df = pd.DataFrame(data, columns = headers) 
    if RP == "Start":
        rpRows = df.loc[(df['RP'] == "RP1") | (df['RP'] == "RP2")]
    elif RP == "Mountain":
        rpRows = df.loc[(df['RP'] == "RP3") | (df['RP'] == "RP4") | (df['RP'] == "RP5") | (df['RP'] == "RP6")]
    elif RP == "Rock":
        rpRows = df.loc[(df['RP'] == "RP7") | (df['RP'] == "RP8")]
    elif RP == "End":
        rpRows = df.loc[(df['RP'] == "RP9") | (df['RP'] == "RP11") | (df['RP'] == "RP13")]
    else:
        rpRows = df.loc[df['RP'] == RP]
    
    totals = [0,0,0,0,0,0,0]
    for index, row in rpRows.iterrows():
        totals[0] = totals[0]+float(row["Anger"])
        totals[1] = totals[1]+float(row["Disgust"])
        totals[2] = totals[2]+float(row["Fear"])
        totals[3] = totals[3]+float(row["Happiness"])
        totals[4] = totals[4]+float(row["Sadness"])
        totals[5] = totals[5]+float(row["Surprise"])
        totals[6] = totals[6]+float(row["Neutral"])
    
    percentageTotal = getPercentageValues(totals)
    
    plt.bar(rpRows.columns[2:],percentageTotal)
    addLabels(percentageTotal, percentageTotal, True)
    plt.yticks(np.arange(0, 110, 10))
    plt.xlabel("Emotions")
    plt.ylabel("Total")
    title = "Total "+RP+" "+Location+" "+str(len(rpRows)) + " participantes"
    plt.title(title)
    plt.savefig("T"+RP+".png")
    plt.show()
    

In [ ]:
def getNumberTime(dataFrame, emotionsDataframe, boxesDataFrame):
    
    numberTime = {}
    for index, row in emotionsDataframe.iterrows():
        rows = boxesDataFrame.loc[boxesDataFrame["Dorsal"] ==row["Dorsal"]]
        imageName = rows.iloc[0]["Image"]
        date = dataFrame.loc[dataFrame["name"] == imageName]
        date = date.iloc[0]["DateTime"]
        
        numberTime[row["Dorsal"]] =  date
        
    return numberTime
    
def timeRPEvolution(emotionsPath, boxesPath, dataPath, RP, Location="?",
                    emotionsNames = ["anger","disgust","fear","happiness","sadness","surprise","neutral"]):
    csv = load_csv(emotionsPath)
    headers = csv[0]
    data = csv[1:]
    
    df = pd.DataFrame(data, columns = headers)
    if RP == "Start":
        emotionsData = df.loc[(df['RP'] == "RP1") | (df['RP'] == "RP2")]
    elif RP == "Mountain":
        emotionsData = df.loc[(df['RP'] == "RP3") | (df['RP'] == "RP4") | (df['RP'] == "RP5") | (df['RP'] == "RP6")]
    elif RP == "Rock":
        emotionsData = df.loc[(df['RP'] == "RP7") | (df['RP'] == "RP8")]
    elif RP == "End":
        emotionsData = df.loc[(df['RP'] == "RP9") | (df['RP'] == "RP11") | (df['RP'] == "RP13")]
    else:
        emotionsData = df.loc[df['RP'] == RP]
    
    csv = load_csv(boxesPath)
    headers = csv[0]
    data = csv[1:]
    
    df = pd.DataFrame(data, columns = headers) 
    if RP == "Start":
        boxesData = df.loc[(df['RP'] == "RP1") | (df['RP'] == "RP2")]
    elif RP == "Mountain":
        boxesData = df.loc[(df['RP'] == "RP3") | (df['RP'] == "RP4") | (df['RP'] == "RP5") | (df['RP'] == "RP6")]
    elif RP == "Rock":
        boxesData = df.loc[(df['RP'] == "RP7") | (df['RP'] == "RP8")]
    elif RP == "End":
        boxesData = df.loc[(df['RP'] == "RP9") | (df['RP'] == "RP11") | (df['RP'] == "RP13")]
    else:
        boxesData = df.loc[df['RP'] == RP]
    
    csv = load_csv(dataPath)
    headers = csv[0]
    data = csv[1:]
    
    df = pd.DataFrame(data, columns = headers) 
    if RP == "Start":
        dataFrame = df.loc[(df['RP'] == "RP1") | (df['RP'] == "RP2")]
    elif RP == "Mountain":
        dataFrame = df.loc[(df['RP'] == "RP3") | (df['RP'] == "RP4") | (df['RP'] == "RP5") | (df['RP'] == "RP6")]
    elif RP == "Rock":
        dataFrame = df.loc[(df['RP'] == "RP7") | (df['RP'] == "RP8")]
    elif RP == "End":
        dataFrame = df.loc[(df['RP'] == "RP9") | (df['RP'] == "RP11") | (df['RP'] == "RP13")]
    else:
        dataFrame = df.loc[df['RP'] == RP]
       
    numberTimeDict = getNumberTime(dataFrame, emotionsData, boxesData)
    
    result = []
    
    for index, row in emotionsData.iterrows():
        time = numberTimeDict[row["Dorsal"]]
        time = datetime.strptime(time, '%Y-%m-%d %H:%M:%S')
        
        current = [row["Dorsal"],
                   time,
                   float(row["Anger"]),
                   float(row["Disgust"]),
                   float(row["Fear"]),
                   float(row["Happiness"]),
                   float(row["Sadness"]),
                   float(row["Surprise"]),
                   float(row["Neutral"])]
        result.append(current)
    result = sorted(result)
    
    df = pd.DataFrame(result, columns=["Dorsal","Time","anger","disgust","fear","happiness","sadness","surprise","neutral"])
    dataFrameSave(df, "./EndData.csv")
        
    time = [r[0] for r in result]
    emotions = [r[1:] for r in result]
    minTime = min(time)
    maxTime = max(time)
    print("Min: ", minTime)
    print("Max: ", maxTime)
    
    answer = ""
    while not answer.isnumeric():
        answer = input("Time interval?(numeric))")
    
    minTime = minTime-timedelta(minutes=float(answer)/10)
    maxTime = maxTime+timedelta(minutes=float(answer)/10)

                                
    fig = plt.figure(figsize=(15,15))
    ax = fig.add_subplot()
    ax.xaxis.set_major_locator(mdates.MinuteLocator(interval=int(answer)))
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
    
    if RP == "salida":
        titleText = "Emotions Evolution"
    else:
        titleText = RP+" Emotions evolution"
    
    ax.set(xlabel = "Time",
           ylabel = "Facial expression strength",
           title = titleText, xlim=[minTime,maxTime], ylim=[0,1])
    ax.plot(time,emotions,label=emotionsNames, linewidth=4)
    plt.legend(loc=0,fontsize='x-large')
    fig.autofmt_xdate()
    plt.grid()
    plt.savefig("E"+RP+".png")
    plt.show()

In [ ]:
if __name__ == "__main__":
    arguments = statisticsExtractionsArguments()
    print(arguments)
    
    answer = ""
    while answer != "y" and answer != "n":
        answer = input("Do you want to see time stats?(y/n))")
    
    while answer == "y":
        answer = ""
        while not answer.isnumeric() and answer != "Start" and answer != "Mountain" and answer != "Rock" and answer != "End":
            answer = input("What RP?(2,3,4,5,6,7,8,9,11,13,Start, Mountain, Rock, End))")
        
        if not answer.isnumeric():
            print(answer)
            timeRPEvolution(arguments["dataFolder"]+arguments["emotionsDataCsvName"], 
                        arguments["dataFolder"]+arguments["boxesDataCsvName"],
                        arguments["dataFolder"]+arguments["sortedDataCsvName"],
                        answer)
        else:
            timeRPEvolution(arguments["dataFolder"]+arguments["emotionsDataCsvName"], 
                        arguments["dataFolder"]+arguments["boxesDataCsvName"],
                        arguments["dataFolder"]+arguments["sortedDataCsvName"],
                        "RP"+answer)
        answer = input("Another one?(y)")
        
    answer = input("Do you want to see average statistics, max or totals?(a,m,t,(q for skip))")
    
    if answer.find("q") != -1:
        sys.exit()    
    if answer.find("a") !=- 1:
        print("\nAVERAGE EMOTIONS\n")
        averageEmotions(arguments["dataFolder"]+arguments["emotionsDataCsvName"], arguments["RPNumberLocations"])
    if answer.find("m") != -1:
        print("\nMAX EMOTIONS\n")
        maxEmotions(arguments["dataFolder"]+arguments["emotionsDataCsvName"], arguments["RPNumberLocations"])
    if answer.find("t") !=- 1:
        print("\nTOTALS EMOTIONS\n")
        totalEmotions(arguments["dataFolder"]+arguments["emotionsDataCsvName"], arguments["RPNumberLocations"])